# Query Suggestion Exploration

## Import Libraries

In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
import numpy as np
import re
import string
import time as timer
regex = re.compile('[%s]' % re.escape(string.punctuation))

## Import query log data

In [2]:
files = ['data/Clean-Data-01.txt', 'data/Clean-Data-02.txt', 'data/Clean-Data-03.txt', 'data/Clean-Data-04.txt', 'data/Clean-Data-05.txt']

In [3]:
ql = pd.concat([pd.read_csv(f, sep='\t') for f in files])

In [4]:
ql = ql.sort_values(by=['AnonID', 'QueryTime'])

In [5]:
ql = ql.reset_index(drop=True)

In [6]:
ql = ql.dropna()

In [7]:
ql['QueryTime'] = pd.to_datetime(ql['QueryTime'])

## Define some constants

For the given formula, we need to find values of some constants

### max_freq

Maximum frequency of any query in QL

In [8]:
max_freq_list = ql['Query'].value_counts()

In [9]:
max_freq = max_freq_list.max()

### max_session_length

In [10]:
max_session_length = 0
'''
for s in ql['AnonID'].unique():
    session = ql[ql['AnonID'] == s]
    length = session.iloc[len(session)-1]['QueryTime'] - session.iloc[0]['QueryTime']
    length =  length / np.timedelta64(1, 's')
    if length > max_session_length:
        max_session_length = length
'''
max_session_length = 7946741.0

## Pre-process some stuff

### Inverted index

Maps from queries to the index of the query log

In [11]:
queryInvertedIndex = {}
def createIndex(ngram, index, id, time):
    global queryInvertedIndex
    if ngram in queryInvertedIndex:
        queryInvertedIndex[ngram].append((index,id,time))
        return 1
    else:
        queryInvertedIndex[ngram] = [(index,id,time)]
        return 0

In [12]:
ql.apply(lambda x: createIndex(str(x['Query']),x.name,x['AnonID'],x['QueryTime']), axis=1)
print(" ")

### QueryCandidates

Maps from a query to all candidates that contain the query + 1 word

In [13]:
queryCandidates = {}
def createCandidates(q, cq):
    global queryCandidates
    if q == cq:
        return
    if q in queryCandidates:
        queryCandidates[q].add(cq)
    else:
        queryCandidates[q] = set([cq])

In [14]:
ql.apply(lambda x: createCandidates(str(x['Query']).rsplit(' ', 1)[0], str(x['Query'])) , axis=1)
print(" ")

## Creating the Query Ranking Score Function

In [15]:
def Freq(CQ):
    return len(queryInvertedIndex[CQ])/max_freq

In [24]:
q_sessions = {}
CQ_sessions = {}
common_sessions = {}
def Mod(CQ, q):
    global q_sessions
    global CQ_sessions
    global common_sessions
    q_sessions = {}
    
    start_time = timer.time()
    if q in queryInvertedIndex:
        #print(len(queryInvertedIndex[q]))
        for query in queryInvertedIndex[q]:
            q_sessions[query[1]] = query[2]
    #print("1: " + str(timer.time()-start_time))     
    if len(q_sessions) == 0:
        return 0
    
    start_time = timer.time()    
    CQ_sessions = {}
    if CQ in queryInvertedIndex:
        for query in queryInvertedIndex[CQ]:
            if query[1] not in CQ_sessions:
                CQ_sessions[query[1]] = query[2]
    #print("2: " + str(timer.time()-start_time))     

    common_sessions = q_sessions.keys() & CQ_sessions.keys()

    common_sessions = [session for session in common_sessions if (CQ_sessions[session]-q_sessions[session])/np.timedelta64(1, 's') > 0]
    return len(common_sessions)/len(q_sessions)

In [17]:
def Time(CQ, q):
    if len(common_sessions) == 0:
        return 0
    min_time = 1000000000
    for session in common_sessions:
        time_diff = (CQ_sessions[session]-q_sessions[session])/np.timedelta64(1, 's')
        if time_diff < min_time:
            min_time = time_diff
    return min_time/max_session_length

In [18]:
def Score(CQ, q):
    freq = Freq(CQ)
    mod = Mod(CQ, q)
    time = Time(CQ, q)
    val = (freq+mod+time)/(1-min([freq, mod, time]))
    return val

In [19]:
def GetCandidates(q, n=10):
    q=regex.sub('', q.lower())
    candidate_scores = {}
    if q in queryCandidates:
        for CQ in queryCandidates[q]:
            candidate_scores[CQ] = Score(CQ, q)
        return sorted(candidate_scores, key=candidate_scores.get, reverse=True)[:n]
    else:
        return []

In [54]:
GetCandidates("the truth about")

KeyError: 4779880

In [138]:
GetCandidates('') 

['ron white',
 'ron jeremy',
 'ron thompson',
 'ron darling',
 'ron gant',
 'ron paul',
 'ron leonard',
 'ron brown',
 'ron john',
 'ron herman']

In [21]:
Mod('google translator', 'google')

83677
1: 0.06191897392272949
2: 0.0


2.9304888055327628e-05

In [158]:
queryInvertedIndex["google"]

[(83, 309, Timestamp('2006-03-18 16:59:05')),
 (157, 507, Timestamp('2006-05-10 11:12:32')),
 (706, 1410, Timestamp('2006-05-01 21:40:54')),
 (744, 2005, Timestamp('2006-03-24 21:25:10')),
 (790, 2178, Timestamp('2006-03-27 20:58:44')),
 (796, 2178, Timestamp('2006-04-11 11:06:20')),
 (809, 2178, Timestamp('2006-05-16 10:54:39')),
 (845, 2421, Timestamp('2006-05-04 15:39:25')),
 (848, 2421, Timestamp('2006-05-04 21:14:33')),
 (849, 2421, Timestamp('2006-05-05 16:16:01')),
 (895, 2708, Timestamp('2006-03-03 20:48:50')),
 (896, 2708, Timestamp('2006-03-04 08:41:35')),
 (897, 2708, Timestamp('2006-03-04 11:59:21')),
 (901, 2708, Timestamp('2006-03-04 21:10:08')),
 (904, 2708, Timestamp('2006-03-05 12:12:10')),
 (907, 2708, Timestamp('2006-03-05 13:18:41')),
 (912, 2708, Timestamp('2006-03-05 15:15:46')),
 (927, 2708, Timestamp('2006-03-11 14:53:31')),
 (938, 2708, Timestamp('2006-03-12 18:01:04')),
 (942, 2708, Timestamp('2006-03-13 22:04:00')),
 (953, 2708, Timestamp('2006-03-16 07:13:25